# Optimizing the Stretched Period Echelle using Peakbagged Modes
## This is Step 2 in our method of characterizing the mixed modes of low-luminosity giants.

### Loading dependencies...

In [13]:
import os, sys, utils
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import pandas as pd
import scipy.stats as stats
import time as timer
import sloscillations.frequencies

from astropy.timeseries import LombScargle
from scipy.stats import qmc
from sklearn.cluster import OPTICS, cluster_optics_dbscan
from sklearn.neighbors import KDTree
from turbo.turbo import Turbo1, TurboM
from turbo.turbo.utils import from_unit_cube, latin_hypercube, to_unit_cube
from scipy.spatial.distance import cdist
from sloscillations import mixed_modes_utils, frequencies

### Loading in helper functions needed to process the spectrum.

In [3]:
#' It is assumed that DeltaNu is in μHz
def DeltaPi1_from_DeltaNu_RGB(DeltaNu):
    # Compute Period spacing (in s) from deltanu
    return 60 + 1.7*DeltaNu

def Lor_model(pds, peak):
    return peak.height / (1 + ((pds.frequency.values - peak.frequency)/peak.linewidth)**2)

def sinc2_model(pds, peak):
    deltanu = np.mean(np.diff(pds.frequency.values))
    return peak.height * np.sinc((pds.frequency.values - peak.frequency)/deltanu)**2

def fit_model(pds, peaks):

    model = np.ones_like(pds.frequency.values)

    for i in range(len(peaks)):
        if np.isfinite(peaks.linewidth.iloc[i]):
            model += Lor_model(pds, peaks.iloc[i,])
        else:
            model += sinc2_model(pds, peaks.iloc[i, ])
    return model

def prepare_l1_peaks(peaks: pd.DataFrame, summary: pd.DataFrame,
                     AIC_cut: [float] = 0.0, height_cut: [float] = 0.0) -> pd.DataFrame:
    """
    Extract the mixed modes from the peaks dataframe.
    
    Parameters
    ----------
    peaks: pd.DataFrame
        Dataframe containing the detected peaks and parameters.
        
    summary: pd.DataFrame
        Dataframe containing the global stellar information.
    
    AIC_cut: Optional[float] = 0.0
        Cut to make in the Akaike Information Criterion if desired.
        
    height_cut: Optional[float] = 0.0
        Cut to make in the mode height if desired.
        
    Outputs
    -------
    pd.DataFrame
        Dataframe containing the mixed mode peaks and associated mode parameters.
    """
    peaks['x'] = ((peaks['frequency'] % summary['DeltaNu'].values - summary['eps_p'].values) / summary['DeltaNu'].values) % 1
    # Don't want to include any modes near l=0 or 2s, this is why this and the step in the next cell is performed.
    x_range = [(np.minimum(np.min(peaks.loc[peaks['l'] == 0, 'x']), np.min(peaks.loc[peaks['l'] == 2, 'x'])) - 0.05) % 1,
               (np.maximum(np.max(peaks.loc[peaks['l'] == 0, 'x']), np.max(peaks.loc[peaks['l'] == 2, 'x'])) + 0.05) % 1]
    
    l1_peaks = peaks.loc[(peaks.l == 1) | ~np.isfinite(peaks.l) | (peaks.l == 3)]
    l1_peaks['x'] = ((l1_peaks['frequency'] % summary['DeltaNu'].values - summary['eps_p'].values) / summary['DeltaNu'].values) % 1
    if x_range[0] < x_range[1]:
        l1_peaks = l1_peaks.loc[(l1_peaks['x'] < x_range[1]) | (l1_peaks['x'] > x_range[0]), ] # changed to OR for HeB
    else:
        print(x_range)
        l1_peaks = l1_peaks.loc[(l1_peaks['x'] > x_range[1]) & (l1_peaks['x'] < x_range[0]), ]


    l1_peaks = l1_peaks.loc[(l1_peaks['height'] > height_cut), ]
    l1_peaks = l1_peaks.loc[(l1_peaks['AIC'] > AIC_cut), ]

    return l1_peaks

### Loading in data. Feel free to modify the KICID variable to any other star included in this repository.

In [20]:
kicx = 3973247

data_folder = os.getcwd() + '/peakbag/intermediate/00%d/' %kicx

summary = pd.read_csv(data_folder + 'summary.csv')
pds = pd.read_csv(data_folder + 'pds_bgr.csv')
peaks = pd.read_csv(data_folder + 'peaksMLE.csv')

# Only keep pds around oscillations
pds = pds.loc[abs(pds['frequency'].values - summary['numax'].values) < 3 * summary['sigmaEnv'].values, ]

# Read in and filter peaks file to be within +/-3 sigmaEnv of numax
peaks = peaks.loc[abs(peaks.frequency.values - summary.numax.values) < 3*summary.sigmaEnv.values, ]
prepped_l1_peaks = prepare_l1_peaks(peaks, summary=summary, AIC_cut=10)

# Optional: weighting the importance of each peak by their relative amplitude. 
# Here we try to weight the low-amplitude mixed modes more.
prepped_l1_peaks['weights'] = (prepped_l1_peaks['amplitude']/np.sum(prepped_l1_peaks['amplitude']))*1000.

[0.9861900778160002, 0.029366167958541745]


<ipython-input-3-ec4d195fd1d7>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  l1_peaks['x'] = ((l1_peaks['frequency'] % summary['DeltaNu'].values - summary['eps_p'].values) / summary['DeltaNu'].values) % 1


### As in Step 1, we pass the peakbagged frequencies to the Frequencies class.

In [22]:
l023_peaks = peaks.loc[(peaks.l == 0) | (peaks.l == 2) ]
pds_l023_removed = pds.assign(power = pds.power / fit_model(pds, l023_peaks))

freqs = frequencies.Frequencies(frequency=pds_l023_removed.frequency.values,
                                numax=summary.numax.values, 
                                delta_nu=summary.DeltaNu.values if np.isfinite(summary.DeltaNu.values) else None, 
                                epsilon_p=summary.eps_p.values if np.isfinite(summary.eps_p.values) else None,
                                alpha=summary.alpha.values if np.isfinite(summary.alpha.values) else None)

### In this step, we are using the dipole ($l=1$) modes and trying to search for a combination of ($\Delta\Pi, q, \epsilon_g, \delta\nu_{\mathrm{rot}}$) that fits the mixed-mode pattern. 

### We will once again use TuRBO, but we will adopt the TuRBO-m algorithm that is more efficient in searching in higher-dimensional spaces.

We have two separate objective functions depending or not we anticipate the presence of core rotational splitting. 
* `freq_tau_rot`: You can specify whether you choose to fit two ($m=-1, 1$) or three ($m=-1,0,1$) rotational components. If not known a priori, fitting for three is the recommended option to find a good fit.
* `freq_tau_rot_singlet`: If you do not anticipate the presence of rotational splitting, use this function to fit only for $m=0$

In [10]:
class freq_tau_rot:
    def __init__(self, inp_freqs, init_dpi, rot_lb=0.1, rot_ub=1.0, eps_g_lb = -0.5, eps_g_ub = 0.5,  nb_comp = 3,
                weights=None):
        self.lb = np.array([init_dpi*0.98, 0.05, eps_g_lb, rot_lb])
        self.ub = np.array([init_dpi*1.02, 0.4, eps_g_ub, rot_ub]) 
        self.nb_comp = nb_comp
        self.weights = weights
        
    def __call__(self, x):
        assert len(x) == 4
        assert x.ndim == 1
        assert np.all(x <= self.ub) and np.all(x >= self.lb)
        
        dpi1, coupling, eps_g, split = x[0], x[1], x[2], x[3]
           
        params = {'calc_l0': True, # copy of params with DPi1 set to candidate dpi from loop
            'calc_l2': True, 
            'calc_l3': False, 
            'calc_nom_l1': True, 
            'calc_mixed': True, 
            'calc_rot': False,
            'DPi1': dpi1,
            'coupling': coupling, 
            'eps_g': eps_g, 
            'split_core': split,
            'split_env': 0.0,
            'l': 1,
            }
    
        freqs(params)

        freqs.generate_tau_values()

        # Compute tau from the zeta value just computed
        real_tau = mixed_modes_utils.peaks_stretched_period(inp_freqs, 
                                                                 freqs.frequency, 
                                                                 freqs.tau)

        freqs_p1 = freqs.l1_mixed_freqs + freqs.l1_zeta * split
        freqs_n1 = freqs.l1_mixed_freqs - freqs.l1_zeta * split

        tau_p1 = mixed_modes_utils.peaks_stretched_period(
            freqs_p1, freqs.frequency, freqs.tau)
        tau_n1 = mixed_modes_utils.peaks_stretched_period(freqs_n1, freqs.frequency, freqs.tau)

        model_freqs = np.c_[freqs.l1_mixed_freqs, freqs_p1, freqs_n1]
        model_tau = np.c_[freqs.l1_mixed_tau, tau_p1, tau_n1]   


        if self.nb_comp == 2:
            X = np.c_[np.r_[model_tau[:,1] - freqs.shift * freqs.DPi1, 
                            model_tau[:,2] - freqs.shift * freqs.DPi1], 
                      np.r_[model_freqs[:,1], 
                            model_freqs[:,2]]]
        else:
            X = np.c_[np.r_[model_tau[:,0],
                            model_tau[:,1] - freqs.shift * freqs.DPi1, 
                            model_tau[:,2] - freqs.shift * freqs.DPi1], 
                      np.r_[model_freqs[:,0],
                            model_freqs[:,1], 
                            model_freqs[:,2]]]


        y_real = (real_tau - freqs.DPi1*(freqs.shift))
        X_real = np.c_[y_real, inp_freqs]
        
        # now compute distance between original frequencies with the frequencies under a new model hypothesis

        c1 = np.vstack((X[:,0]/freqs.DPi1, (X[:,1]-freqs.numax)/freqs.delta_nu)).T
        c2 = np.vstack((X_real[:,0]/freqs.DPi1, (X_real[:,1]-freqs.numax)/freqs.delta_nu)).T
        tree = KDTree(c1, metric='euclidean')
        nearest_dists, nearest_idx = tree.query(c2, k=1)
        
        if self.weights is not None:
            return np.median(nearest_dists*self.weights)
        else:
            return np.median(nearest_dists)

        
        

class freq_tau_rot_singlet:
    def __init__(self, inp_freqs, init_dpi, eps_g_lb = -0.5, eps_g_ub = 0.3, weights = None):
        self.lb = np.array([init_dpi*0.98, 0.05, eps_g_lb])
        self.ub = np.array([init_dpi*1.02, 0.4, eps_g_ub])
        self.weights = weights

    def __call__(self, x):
        assert len(x) == 3
        assert x.ndim == 1
        assert np.all(x <= self.ub) and np.all(x >= self.lb)
        
        dpi1, coupling, eps_g = x[0], x[1], x[2]
         
        params = {'calc_l0': True, # copy of params with DPi1 set to candidate dpi from loop
            'calc_l2': True, 
            'calc_l3': False, 
            'calc_nom_l1': True, 
            'calc_mixed': True, 
            'calc_rot': False,
            'DPi1': dpi1,
            'coupling': coupling, 
            'eps_g': eps_g, 
            'split_core': 0.0,
            'split_env': 0.0,
            'l': 1,
            }
    
        freqs(params)
        freqs.generate_tau_values()

        # Compute tau from the zeta value just computed
        real_tau = mixed_modes_utils.peaks_stretched_period(inp_freqs, 
                                                                 freqs.frequency, 
                                                                 freqs.tau)


        model_freqs = freqs.l1_mixed_freqs.reshape(-1,1)
        model_tau = freqs.l1_mixed_tau.reshape(-1,1)

        X = np.c_[model_tau[:,0], model_freqs[:,0]]    
        y_real = (real_tau - freqs.DPi1*(freqs.shift))
        X_real = np.c_[y_real, inp_freqs]

        ## KDTree ##
        c1 = np.vstack((X[:,0]/freqs.DPi1, (X[:,1]-freqs.numax)/freqs.delta_nu)).T
        c2 = np.vstack((X_real[:,0]/freqs.DPi1, (X_real[:,1]-freqs.numax)/freqs.delta_nu)).T
        tree = KDTree(c1, metric='euclidean')
        nearest_dists, nearest_idx = tree.query(c2, k=1)
        
        if self.weights is not None:
            return np.median(nearest_dists*self.weights)
        else:
            return np.median(nearest_dists)

### The inputs into the objective function concern the range over which to search:

`inp_freqs`: The dipole mode frequencies that we wish to match to a model of the stretched period echelle diagram.

`init_dpi`: The value of $\Delta\Pi$ over which we will search over. Searching over a more restricted but informed range of $\Delta\Pi$ greatly improves the likelihood of finding a good solution. Ideally, this will be set to the optimal $\Delta\Pi$ from the PSxPS from Part 1. For this demo, we will use the $\Delta\Pi$ inferred from $\Delta\nu$.

`rot_lb`: $\delta\nu_{\mathrm{rot}}$ lower bound. Default: 0.1

`rot_ub`: $\delta\nu_{\mathrm{rot}}$ upper bound. Default: 1.0

`eps_g_lb`: $\epsilon_g$ lower bound. Default: -0.5

`eps_g_ub`: $\epsilon_g$ upper bound. Default: 0.5

`nb_comp`: Only pertinent to `freq_tau_rot`. Number of rotationally split components (2 or 3). Default: 3.

`weights`: Weights for each mixed mode. Default: None (i.e. unweighted). 

In [42]:
f_rot = freq_tau_rot(inp_freqs = prepped_l1_peaks['frequency'].values,# generic case
    init_dpi=DeltaPi1_from_DeltaNu_RGB(freqs.delta_nu)[0],
    rot_lb=0.05, 
    rot_ub=1.0,
    eps_g_lb = -0.05, 
    eps_g_ub =0.35, 
    nb_comp = 3, 
    weights=(prepped_l1_peaks['amplitude'].values/np.sum(prepped_l1_peaks['amplitude'].values))*1000.) 

# f_rot = freq_tau_rot_singlet( # fitting only m=0
#     init_dpi = DeltaPi1_from_DeltaNu_RGB(freqs.delta_nu), 
#     eps_g_lb = -0.5,
#     eps_g_ub =0.35, 
#     weights=(prepped_l1_peaks['amplitude'].values/np.sum(prepped_l1_peaks['amplitude'].values))*1000.) 

### Now proceed with optimization using TuRBO-m. Since the space is now higher-dimensional compared to Step 1, we will now run over 5000 iterations.

In [43]:
# Turbo M
turbo1_rot = TurboM(
    f=f_rot,  # Handle to objective function
    lb=f_rot.lb,  # Numpy array specifying lower bounds
    ub=f_rot.ub,  # Numpy array specifying upper bounds
    n_init=50,  # Number of initial bounds from an Symmetric Latin hypercube design
    max_evals=5000,  # Maximum number of evaluations
    n_trust_regions=10,  # Number of trust regions
    batch_size=100,  # How large batch size TuRBO uses
    verbose=True,  # Print information from each batch
    use_ard=True,  # Set to true if you want to use ARD for the GP kernel
    max_cholesky_size=2000,  # When we switch from Cholesky to Lanczos
    n_training_steps=50,  # Number of steps of ADAM to learn the hypers
    min_cuda=64,  # Run on the CPU for small datasets
    device="cuda",  # "cpu" or "cuda"
    dtype="float64",  # float64 or float32
)
# turbo1_rot.failtol = 20 # double the tolerance
turbo1_rot.n_cand = 5000

Using dtype = torch.float64 
Using device = cuda


In [ ]:
init_time = timer.time()
turbo1_rot.optimize()
print('Elapsed Time: ', timer.time()-init_time)

TR-0 starting from: 0.7605
TR-1 starting from: 0.6052
TR-2 starting from: 0.8629
TR-3 starting from: 0.7989
TR-4 starting from: 0.847
TR-5 starting from: 0.6471
TR-6 starting from: 0.513
TR-7 starting from: 0.8737
TR-8 starting from: 0.7596
TR-9 starting from: 0.801
800) New best @ TR-7: 0.3655, Best Sol: [81.24439736  0.15669165  0.10937384  0.35654226]
900) New best @ TR-7: 0.3487, Best Sol: [8.13320099e+01 1.61855645e-01 6.07940479e-02 4.22633054e-01]
1000) New best @ TR-7: 0.2752, Best Sol: [8.13064422e+01 1.19359569e-01 6.60456711e-02 3.76863689e-01]
1200) New best @ TR-7: 0.2139, Best Sol: [8.13375877e+01 1.42076700e-01 3.34233657e-02 4.06938969e-01]
1500) New best @ TR-7: 0.1992, Best Sol: [8.13112996e+01 1.43893250e-01 5.92912630e-02 4.07717470e-01]
1600) New best @ TR-7: 0.1835, Best Sol: [81.28850341  0.14216652  0.0816418   0.39237106]
1900) New best @ TR-7: 0.1748, Best Sol: [81.27678653  0.14557301  0.09218362  0.40283804]
2200) TR-7 converged to: : 0.1748
